In [3]:
import pyspark
from delta import *
from delta.tables import *
from pyspark.sql.functions import *

builder = pyspark.sql.SparkSession.builder.appName("demo").master("spark://spark-master:7077") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") \
    .config("spark.sql.extensions","io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog","org.apache.spark.sql.delta.catalog.DeltaCatalog") \
    .config("spark.hadoop.fs.s3a.access.key","datalake") \
    .config("spark.hadoop.fs.s3a.secret.key","datalake") \
    .config("spark.hadoop.fs.s3a.endpoint","http://minio:9000") \
    .config("spark.hadoop.fs.s3a.path.style.access", "true") \
    .config("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem")

spark = configure_spark_with_delta_pip(builder).enableHiveSupport().getOrCreate()

RuntimeError: Java gateway process exited before sending its port number

In [2]:
spark

In [74]:
df_userBronze = spark.read.json('s3a://camada-bronze/user')

In [75]:
df_userBronze.count()

82

In [77]:
df_userSimple = df_userBronze.select('first_name','last_name','id','username')

In [79]:
df_userSimple.repartition(4).write.parquet('s3a://camada-prata/user',mode='overwrite')

In [55]:
df_userPrata = spark.read.parquet('s3a://camada-prata/user')

In [56]:
df_userPrata.count()

67

In [38]:
#df_userPrata.write.format('delta').save('s3a://camada-ouro/user')

In [57]:
df_userDelta = DeltaTable.forPath(spark,'s3a://camada-ouro/user')

In [58]:
df_userDelta.toDF().count()

51

In [41]:
type(df_userDelta)

delta.tables.DeltaTable

In [62]:
df_userDelta.alias('ouro') \
    .merge(df_userPrata.alias('prata'),
    "ouro.id = prata.id") \
    .whenMatchedUpdate(set = {"username": col("prata.username")}) \
    .whenNotMatchedInsert(values = {"id": col("prata.id"),"username": col("prata.username")}) \
    .execute()

In [63]:
df_userDelta = DeltaTable.forPath(spark,'s3a://camada-ouro/user')
df_userDelta.toDF().count()

67

In [64]:
df_userDelta.toDF().show()

+----------+---------+----+----------------+
|first_name|last_name|  id|        username|
+----------+---------+----+----------------+
|   Emanuel|    Marks| 202|   emanuel.marks|
|    Samual|    Crist| 245|    samual.crist|
|      null|     null| 251|     angelo.koss|
|      null|     null| 358|    cyrus.casper|
|     Ozell|  Pacocha| 391|   ozell.pacocha|
|      null|     null| 801|       mason.rau|
|    Tomika|     Mohr| 867|     tomika.mohr|
|      Otto|     Kris| 931|       otto.kris|
|      null|     null|1230|theodore.ritchie|
|       Lou|  Watsica|1524|     lou.watsica|
|      null|     null|1746|   horace.turner|
|    Luanne|    Koepp|1801|    luanne.koepp|
|      null|     null|2083|  mason.predovic|
|      null|     null|2534|adalberto.dooley|
|   Charlie|  Johnson|2572| charlie.johnson|
|   Zenobia|   Parker|2837|  zenobia.parker|
|      null|     null|2839|  jules.mitchell|
|    Roxane|  Gerlach|2861|  roxane.gerlach|
|    Colton| Franecki|2867| colton.franecki|
|      nul

In [67]:
df_userSimple.createOrReplaceTempView('tab')

In [71]:
df_test = spark.sql('select * from tab where id > 7300')

In [72]:
df_test.show()

UsageError: Line magic function `%sql` not found.


In [80]:
help(df_test.filter)

Help on method filter in module pyspark.sql.dataframe:

filter(condition: 'ColumnOrName') -> 'DataFrame' method of pyspark.sql.dataframe.DataFrame instance
    Filters rows using the given condition.
    
    :func:`where` is an alias for :func:`filter`.
    
    .. versionadded:: 1.3.0
    
    Parameters
    ----------
    condition : :class:`Column` or str
        a :class:`Column` of :class:`types.BooleanType`
        or a string of SQL expression.
    
    Examples
    --------
    >>> df.filter(df.age > 3).collect()
    [Row(age=5, name='Bob')]
    >>> df.where(df.age == 2).collect()
    [Row(age=2, name='Alice')]
    
    >>> df.filter("age > 3").collect()
    [Row(age=5, name='Bob')]
    >>> df.where("age = 2").collect()
    [Row(age=2, name='Alice')]



In [81]:
import requests

In [82]:
r = requests.get('https://random-data-api.com/api/v2/users')

In [83]:
r.json()

{'id': 7537,
 'uid': '89c0ff87-d75c-4fbb-88ad-f4ff53f76abc',
 'password': 'mFTXCrOS9H',
 'first_name': 'Jamison',
 'last_name': 'Fisher',
 'username': 'jamison.fisher',
 'email': 'jamison.fisher@email.com',
 'avatar': 'https://robohash.org/quiaquirem.png?size=300x300&set=set1',
 'gender': 'Agender',
 'phone_number': '+1-758 1-521-477-3813',
 'social_insurance_number': '776045551',
 'date_of_birth': '1960-11-12',
 'employment': {'title': 'Direct Consulting Coordinator',
  'key_skill': 'Fast learner'},
 'address': {'city': 'West Marva',
  'street_name': 'Duane Gardens',
  'street_address': '196 Pagac Via',
  'zip_code': '61204-4737',
  'state': 'Illinois',
  'country': 'United States',
  'coordinates': {'lat': 83.53003509882083, 'lng': -164.43744273559022}},
 'credit_card': {'cc_number': '4335-8763-6109-2084'},
 'subscription': {'plan': 'Basic',
  'status': 'Pending',
  'payment_method': 'Cheque',
  'term': 'Monthly'}}

In [92]:
df_json = spark.read.json(spark.sparkContext.parallelize(r.json()))

In [93]:
df_json.show()

+--------------------+
|     _corrupt_record|
+--------------------+
|                  id|
|                 uid|
|            password|
|          first_name|
|           last_name|
|            username|
|               email|
|              avatar|
|              gender|
|        phone_number|
|social_insurance_...|
|       date_of_birth|
|          employment|
|             address|
|         credit_card|
|        subscription|
+--------------------+

